In [1]:
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd

driver = webdriver.Chrome()
driver.minimize_window()

link = 'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile'
driver.get(link)

wait = WebDriverWait(driver, 20)

routelist = []
routelinklist = []

scrolling = True
while scrolling:
    old_page_source = driver.page_source
    body = driver.find_element(By.TAG_NAME, "body")
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    new_page_source = driver.page_source
    if new_page_source == old_page_source:
        scrolling = False

for page_number in range(1, 5):
    try:
        routes = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'route')))
        for route in routes:
            route_text = route.text
            route_link = route.get_attribute('href')
            routelist.append(route_text)
            routelinklist.append(route_link)
        if page_number < 4:
            time.sleep(2)
            next_page_xpath = f'//div[contains(@class,"DC_117_pageTabs") and text()="{page_number + 1}"]'
            next_page = wait.until(EC.element_to_be_clickable((By.XPATH, next_page_xpath)))
            time.sleep(5)
            driver.execute_script("arguments[0].scrollIntoView(true);", next_page)
            actions = ActionChains(driver)
            actions.move_to_element(next_page).click().perform()
    finally:
        print(f"Processed page {page_number}")

route_data = {'Route': routelist, 'Link': routelinklist}
df_routes = pd.DataFrame(route_data)

df_routes.index = df_routes.index + 1
df_routes.to_csv('HRCT_data.csv', index=False)

driver.quit()


Processed page 1
Processed page 2
Processed page 3
Processed page 4


In [2]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome()
master_df = pd.DataFrame()

for link, routename in zip(routelinklist, routelist):
    driver.get(link)

    state = 'HRTC'
    statelist = []
    busroutelist = []
    busnamelist = []
    bustypelist = []
    departing_time_list = []
    duration_list = []
    reaching_time_list = []
    star_rating_list = []
    price_list = []
    seats_available_list = []

    scrolling = True
    last_height = driver.execute_script("return document.body.scrollHeight")

    while scrolling:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)
        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(3)
        if new_height == last_height:
            scrolling = False
        else:
            last_height = new_height

    busname_1 = driver.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
    bustype_1 = driver.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
    departing_time_1 = driver.find_elements(By.XPATH, "//div[@class='dp-time f-19 d-color f-bold']")
    duration_1 = driver.find_elements(By.XPATH, "//div[@class='dur l-color lh-24']")
    reaching_time_1 = driver.find_elements(By.XPATH, "//div[@class='bp-time f-19 d-color disp-Inline']")
    star_rating_1 = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']//span")
    price_1 = driver.find_elements(By.XPATH, "//span[contains(@class, 'f-19')]")
    seats_available_1 = driver.find_elements(By.XPATH, "//div[@class='seat-left m-top-30']")

    for busname, bustype, departing_time, duration_time, reaching_time, star, price, seats in zip(
        busname_1, bustype_1, departing_time_1, duration_1, reaching_time_1, star_rating_1, price_1, seats_available_1):
        busnamelist.append(busname.text)
        bustypelist.append(bustype.text)
        departing_time_list.append(departing_time.text)
        duration_list.append(duration_time.text)
        reaching_time_list.append(reaching_time.text)
        star_rating_list.append(star.text)
        price_list.append(price.text)
        seats_available_list.append(seats.text)
        statelist.append(state)
        busroutelist.append(routename)

    data = {
        'State': statelist,
        'Route': busroutelist,
        'Bus Name': busnamelist,
        'Bus Type': bustypelist,
        'Departing Time': departing_time_list,
        'Duration Time': duration_list,
        'Reaching Time': reaching_time_list,
        'Price': price_list,
        'Seats Available': seats_available_list,
        'Star Rating': star_rating_list
    }

    df = pd.DataFrame(data)
    master_df = pd.concat([master_df, df], ignore_index=True)
    master_df.to_csv('HRCT.csv', index=False)

    print(f"Data from {link} added successfully!")

print("HRCT CSV saved.")
driver.quit()


Data from https://www.redbus.in/bus-tickets/delhi-to-shimla added successfully!
Data from https://www.redbus.in/bus-tickets/chandigarh-to-hamirpur-himachal-pradesh added successfully!
Data from https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-chandigarh added successfully!
Data from https://www.redbus.in/bus-tickets/shimla-to-delhi added successfully!
Data from https://www.redbus.in/bus-tickets/delhi-to-chandigarh added successfully!
Data from https://www.redbus.in/bus-tickets/hamirpur-himachal-pradesh-to-delhi added successfully!
Data from https://www.redbus.in/bus-tickets/chamba-himachal-pradesh-to-chandigarh added successfully!
Data from https://www.redbus.in/bus-tickets/delhi-to-hamirpur-himachal-pradesh added successfully!
Data from https://www.redbus.in/bus-tickets/chandigarh-to-dharamshala added successfully!
Data from https://www.redbus.in/bus-tickets/delhi-to-chamba-himachal-pradesh added successfully!
Data from https://www.redbus.in/bus-tickets/chamba-himachal-p